In [ ]:
# standard python utilities
import os
from os.path import join, basename,dirname, exists, expanduser
import sys
import glob
import pandas as pd
import numpy as np
import time

# standard python plotting utilities
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

# standard geospatial python utilities
# import pyproj # for converting proj4string
# import shapely
import geopandas as gpd
# import rasterio

# mapping utilities
import contextily as ctx
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
from matplotlib.ticker import MaxNLocator


In [ ]:
usr_dir = expanduser('~')
doc_dir = join(usr_dir, 'Documents')
    
# dir of all gwfm data
gwfm_dir = dirname(doc_dir)+'/Box/research_cosumnes/GWFlowModel'
# dir of stream level data for seepage study
proj_dir = gwfm_dir + '/Oneto_Denier/'
dat_dir = proj_dir+'Stream_level_data/'

fig_dir = proj_dir+'/Streambed_seepage/figures/'
hob_dir = join(gwfm_dir, 'HOB_data')
sfr_dir = gwfm_dir+'/SFR_data/'




In [ ]:
def add_path(fxn_dir):
    """ Insert fxn directory into first position on path so local functions supercede the global"""
    if fxn_dir not in sys.path:
        sys.path.insert(0, fxn_dir)

add_path(doc_dir+'/GitHub/flopy')
import flopy 
py_dir = join(doc_dir,'GitHub/CosumnesRiverRecharge/python_utilities')
add_path(py_dir)
from mf_utility import get_dates, get_layer_from_elev, clean_wb
from map_cln import gdf_bnds, plt_cln

# from importlib import reload
# import mf_utility
# reload(mf_utility)

In [ ]:
# scenario specific function
from OD_utility import run_stats

In [ ]:
# scenario = '' # baseline, levee removal occurred in 2014
# create identifier for scenario if levee removal didn't occur
scenario = 'no_reconnection'

In [ ]:
ext_dir = 'F:/WRDAPP'
c_dir = 'C:/WRDAPP'
if os.path.exists(ext_dir):
    loadpth = ext_dir 
elif os.path.exists(c_dir):
    loadpth = c_dir 
loadpth +=  '/GWFlowModel/Cosumnes/Stream_seepage'

upscale = 'upscale4x_'
# model_nam = 'oneto_denier_'+upscale+'2014_2018'
model_nam = 'oneto_denier_'+upscale+'2014_2020'
model_ws = join(loadpth,model_nam)

if scenario != '':
    model_ws += '_' + scenario
    
# model_ws = join(loadpth,'parallel_oneto_denier','realization000')
load_only = ['DIS','UPW','SFR','OC', "EVT",'LAK']
m = flopy.modflow.Modflow.load('MF.nam', model_ws= model_ws, 
                                exe_name='mf-owhm.exe', version='mfnwt',
                              load_only=load_only,
                              )


In [ ]:
nrow, ncol = (m.dis.nrow, m.dis.ncol)

In [ ]:
model_ws0 = join(loadpth,model_nam)

In [ ]:
print('Quantiles: ',[0,0.5,0.6,0.75,1])
print('HK :',np.quantile(m.upw.hk.array,[0,0.5,0.6,0.75,1]))
print('VKA :',np.quantile(m.upw.vka.array,[0,0.5,0.6,0.75,1]))

In [ ]:
model_grp = 'inset_oneto_denier'
grid_dir = join(gwfm_dir, 'DIS_data/streambed_seepage/grid')
grid_fn = join(grid_dir, model_grp,'rm_only_grid.shp')
grid_p = gpd.read_file(grid_fn)
grid_p.crs='epsg:32610'
m_domain = gpd.GeoDataFrame(pd.DataFrame([0]), geometry = [grid_p.unary_union], crs=grid_p.crs)

In [ ]:
XSg = pd.read_csv(join(model_ws,'04_XSg_filled.csv'))
XSg = gpd.GeoDataFrame(XSg, geometry = gpd.points_from_xy(XSg.Easting, XSg.Northing), crs='epsg:32610')

# overwrite SFR segment/reach input relevant to seepage
# sensor_dict = pd.read_csv(join(model_ws, 'sensor_xs_dict.csv'), index_col=0)
# XS_params = sensor_dict.join(params.set_index('Sensor'), on='Sensor')

In [ ]:
params = pd.read_csv(model_ws+'/ZonePropertiesInitial.csv', index_col='Zone')
# convert from m/s to m/d
params['K_m_d'] = params.K_m_s * 86400 
vka = m.upw.vka.array
tprogs_vals = np.arange(1,5)
tprogs_hist = np.flip([0.590, 0.155, 0.197, 0.058])
tprogs_quants = 1-np.append([0], np.cumsum(tprogs_hist)/np.sum(tprogs_hist))
vka_quants = pd.DataFrame(tprogs_quants[1:], columns=['quant'], index=tprogs_vals)
# dataframe summarizing dominant facies based on quantiles
vka_quants['vka_min'] = np.quantile(vka, tprogs_quants[1:])
vka_quants['vka_max'] = np.quantile(vka, tprogs_quants[:-1])
vka_quants['facies'] = params.loc[tprogs_vals].Lithology.values

In [ ]:
sfrdf = pd.DataFrame(m.sfr.reach_data)
grid_sfr = grid_p.set_index(['row','column']).loc[list(zip(sfrdf.i+1,sfrdf.j+1))].reset_index(drop=True)
grid_sfr = pd.concat((grid_sfr,sfrdf),axis=1)
# group sfrdf by vka quantiles
sfr_vka = vka[grid_sfr.k, grid_sfr.i, grid_sfr.j]
for p in vka_quants.index:
    facies = vka_quants.loc[p]
    grid_sfr.loc[(sfr_vka< facies.vka_max)&(sfr_vka>= facies.vka_min),'facies'] = facies.facies
#     # add color for facies plots
# grid_sfr = grid_sfr.join(gel_color.set_index('geology')[['color']], on='facies')

In [ ]:
lak_shp = join(gwfm_dir,'LAK_data/floodplain_delineation')
# shapefile rectangle of the area surrounding the Dam within about 5 cells
lak_gpd = gpd.read_file(join(lak_shp,'LCRFR_ModelDom_2017/LCRFR_2DArea_2015.shp' )).to_crs('epsg:32610')

lak_cells = gpd.sjoin(grid_p,lak_gpd,how='right',predicate='within').drop(columns='index_left')

# filter zone budget for Blodgett Dam to just within 5 cells or so of the Dam
zon_lak = np.zeros((grid_p.row.max(),grid_p.column.max()),dtype=int)
zon_lak[lak_cells.row-1,lak_cells.column-1]=1

zon_mod = np.ones((grid_p.row.max(),grid_p.column.max()),dtype=int)

In [ ]:
zon_color_dict = pd.read_excel('mf_wb_color_dict.xlsx',sheet_name='owhm_wb_dict', header=0, index_col='flux',comment='#').color.to_dict()
zon_name_dict = pd.read_excel('mf_wb_color_dict.xlsx',sheet_name='owhm_wb_dict', header=0, index_col='flux',comment='#').name.to_dict()

zb_alt = pd.read_excel('mf_wb_color_dict.xlsx',sheet_name='flopy_to_owhm', header=0, index_col='flopy',comment='#').owhm.to_dict()


## Sensor data and XS data

In [ ]:
rm_grid = pd.read_csv(join(proj_dir, 'mw_hob_cleaned.csv'))
rm_grid = gpd.GeoDataFrame(rm_grid, geometry = gpd.points_from_xy(rm_grid.Longitude,rm_grid.Latitude), 
                           crs='epsg:4326').to_crs(grid_p.crs)
# get model layer for heads
hob_row = rm_grid.row.values-1
hob_col = rm_grid.column.values-1

In [ ]:
gwl_long = pd.read_csv(join(model_ws,'gwl_long.csv'), parse_dates=['dt'])

In [ ]:
# XS are every 100 m
xs_all = pd.read_csv(dat_dir+'XS_point_elevations.csv',index_col=0)
xs_all = gpd.GeoDataFrame(xs_all,geometry = gpd.points_from_xy(xs_all.Easting,xs_all.Northing), crs='epsg:32610')


In [ ]:

# correspond XS to sensors
rm_elev = gpd.sjoin_nearest(XSg, rm_grid, how='right',lsuffix='xs', rsuffix='rm')
#MW_11, MW_CP1 had doubles with sjoin_nearest due to XS duplicates from Oneto_Denier
rm_elev = rm_elev.drop_duplicates(['xs_num','Sensor'])

## Model output - time variant

In [ ]:
hdobj0 = flopy.utils.HeadFile(model_ws0+'/MF.hds')
hdobj = flopy.utils.HeadFile(model_ws+'/MF.hds')
spd_stp = hdobj.get_kstpkper()
times = hdobj.get_times()


In [ ]:
strt_date, end_date, dt_ref = get_dates(m.dis, ref='strt')


# HOB

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

def nse(targets,predictions):
    return 1-(np.sum((targets-predictions)**2)/np.sum((targets-np.mean(predictions))**2))

def clean_hob(model_ws):
    hobout = pd.read_csv(join(model_ws,'MF.hob.out'),delimiter=r'\s+', header = 0,names = ['sim_val','obs_val','obs_nam'],
                         dtype = {'sim_val':float,'obs_val':float,'obs_nam':object})
    hobout[['Sensor', 'spd']] = hobout.obs_nam.str.split('p',n=2, expand=True)
    hobout['kstpkper'] = list(zip(np.full(len(hobout),0), hobout.spd.astype(int)))
    hobout = hobout.join(dt_ref.set_index('kstpkper'), on='kstpkper')
    hobout.loc[hobout.sim_val.isin([-1e30, -999.99,-9999]), 'sim_val'] = np.nan
    hobout = hobout.dropna(subset='sim_val')
    hobout['error'] = hobout.obs_val - hobout.sim_val
    hobout['sq_error'] = hobout.error**2
    
    return(hobout)


In the final iteration of the model, Oneto-Ag isn't fit that badly anymore likely because of the aquifer thickening and updated Kx, adjusting Ss would improve further perhaps, but since the focus is the shallow network we will leave it out.

In [ ]:
hobout = clean_hob(model_ws)
# removing oneto ag because of large depth offset
hobout = hobout[hobout.Sensor != 'MW_OA']

In [ ]:
hobout0 = clean_hob(model_ws0)
# removing oneto ag because of large depth offset
hobout0 = hobout0[hobout0.Sensor != 'MW_OA']

In [ ]:
def return_stat(hobout):
    r2 = r2_score(hobout.obs_val, hobout.sim_val)
    RMSE = mean_squared_error(hobout.obs_val, hobout.sim_val, squared=False) # false returns RMSE instead of MSE
    NSE = nse(hobout.obs_val, hobout.sim_val)
    return(r2, RMSE, NSE)

In [ ]:
return_stat(hobout0), return_stat(hobout)

In [ ]:
hob_long = hobout.join(hobout0.set_index('obs_nam')[['sim_val']], on='obs_nam',rsuffix='0')
hob_long = hob_long.melt(id_vars=['dt', 'Sensor'],value_vars=['sim_val','obs_val','sim_val0'],
                         value_name='gwe', var_name='type')

# hob_long = hobout.melt(id_vars=['dt', 'Sensor'],value_vars=['sim_val','obs_val'], value_name='gwe', var_name='type')


In [ ]:
# g = sns.relplot(hob_long[hob_long.Sensor.isin(['MW_2','MW_3'])], x='dt',y='gwe', 
#                 row='Sensor',hue = 'type', kind='line')


In [ ]:
# hob_long, x='dt',y='
g = sns.relplot(hob_long, x='dt',y='gwe',col='Sensor',hue = 'type',  col_wrap=4, kind='line')

axes = g.axes.flatten()
mw = hob_long.Sensor.unique()

for n in np.arange(0,len(axes)):
    mw_dat = rm_elev[rm_elev.Sensor ==mw[n]]
    axes[n].axhline(mw_dat['MPE (meters)'].values[0], ls='--', linewidth=3, color='brown')
    axes[n].axhline(mw_dat['z_m_min_cln'].values[0]-1, ls='--', linewidth=3, color='blue')
    # axes[n].axhline(mw_dat['bot_screen_m'].values[0]-1, ls='--', linewidth=3, color='black')

In [ ]:
gage_cols = ['time','stage','volume','conc','inflows','outflows','conductance','error']

def read_gage(gagenam):
    gage = pd.read_csv(gagenam,skiprows=1, delimiter = r'\s+', engine='python')
    cols = gage.columns[1:-1]
    gage = gage.dropna(axis=1)
    gage.columns = cols
    strt_date = pd.to_datetime(m.dis.start_datetime)
    # the lake output includes the initial conditions (time==0)
    gage = gage[gage.Time >0]
    gage['dt'] = strt_date+((gage.Time-1)*24).astype('timedelta64[h]')
    gage = gage.set_index('dt')
    gage['dVolume'] = gage.Volume.diff()
    gage['Total_In'] = gage[['Precip.','Runoff','GW-Inflw','SW-Inflw']].sum(axis=1)
    gage['Total_Out'] = gage[['Evap.','Withdrawal','GW-Outflw','SW-Outflw']].sum(axis=1)
    gage['In-Out'] = gage.Total_In - gage.Total_Out
    # remove the steady state depth (dry start)
    gage.loc[gage['Stage(H)']<gage['Stage(H)'].quantile(0.01), 'Stage(H)'] = gage['Stage(H)'].quantile(0.01)
    # approximate depth, may not always work
    gage['depth'] = gage['Stage(H)']- gage['Stage(H)'].quantile(0.05)
    gage.loc[gage.depth<0,'depth']= 0
#     gage['name'] = run
    return(gage)


In [ ]:
lak_out0 = read_gage(join(model_ws0, 'MF_lak.go'))
lak_out = read_gage(join(model_ws, 'MF_lak.go'))


In [ ]:
# the days with WB error of 100% actually have no issues in the water budget
# lak_out[lak_out['Percent-Err']==100]

In [ ]:
# there are about 40 days with about 100% error or -60% error, the rest are well below 1%
# the error seems to happen when there is very little flow in the system rather than too much
# lak_out.plot(y='Percent-Err')
# cols = ['Percent-Err','GW-Outflw','SW-Inflw','SW-Outflw', 'In-Out']
# fig,ax = plt.subplots(len(cols),1,sharex=True)
# for n, col in enumerate(cols):
#     lak_out.plot(y=col, ax=ax[n])
# lak_out.columns
# plt.ylim(-1, 1)

Explain the mechanism of the results:
- a lower flow threshold and greater flow fraction puts more flow onto the floodplain so there is greater depth
- greater depth and area increases floodplain recharge
- the groundwater inflow is very small (1E3) compared to the outflow (1E5)
- would be good to plot groundwater elevation as well to highlight why the groundwater outflow changes

In [ ]:
# find where lake existed
lak_lay, lay_row, lak_col = np.where(m.lak.lakarr.array[0]==1)
lak_kij = pd.DataFrame(np.transpose(np.where(m.lak.lakarr.array[0]==1)), columns=['k','i','j'])
# get first layer below lake cells
lak_kij = (lak_kij.groupby(['i','j']).max()+1).reset_index()
# create tuples for sampling
lak_idx = list(zip(lak_kij.k, lak_kij.i, lak_kij.j))

In [ ]:
def get_lak_head(hdobj, lak_idx):
    # get heads under the lake
    lak_ts = hdobj.get_ts(lak_idx)
    lak_ts_df = pd.DataFrame(lak_ts, columns=['totim']+lak_idx)
    lak_ts_df = lak_ts_df.set_index('totim')
    lak_ts_df = lak_ts_df.melt(ignore_index=False)
    lak_ts_df[['k','i','j']] = lak_ts_df.variable.tolist()
    lak_ts_df = lak_ts_df.drop(columns='variable') # drop to speed up groupby
    lak_head = lak_ts_df.groupby(['totim','i','j']).max().groupby('totim').mean()
    return lak_head

In [ ]:
lak_head = get_lak_head(hdobj, lak_idx)
lak_head0 = get_lak_head(hdobj0, lak_idx)

In [ ]:
import matplotlib.ticker as mtick


In [ ]:
cols = ['SW-Inflw', 'Stage(H)','GW-Outflw']
labels=['SW Inflow \n(million $m^3/d$)', 'Stage (m)', 'GW Outflow \n(million $m^3/d$)']

scale = 1E6
fig,ax = plt.subplots(len(cols)+1,1, figsize=(6.5,6.5), dpi=300, sharex=True)
for n, col in enumerate(cols):
    if col in ['SW-Inflw','GW-Outflw']:
        scale = 1E6
    else:
        scale = 1
    lak_out0[col].multiply(1/scale).plot(y=col, ax=ax[n], legend=False, label='Baseline')
    lak_out[col].multiply(1/scale).plot(y=col, ax=ax[n],legend=False, alpha=0.7, label='No Reconnection')
    ax[n].set_ylabel(labels[n])

# ax[0].yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1e'))
# ax[2].yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1e'))

ax[-1].plot(lak_out.index, lak_head0['value'].values)
ax[-1].plot(lak_out.index, lak_head['value'].values,alpha=0.7)
ax[-1].set_ylabel('GW Elevation (m)')
# fig.legend(['Restoration','Baselien'], ncol=2, loc='outside upper center', bbox_to_anchor=(0.5, 1.05),)
ax[0].legend(['Restoration','Baseline'], ncol=2, loc='upper right')
fig.tight_layout(h_pad=-0.1)
plt.xlabel('Datetime')

## Output reference values  
We need to supply numeric values to use in the result section to support comments on the differences between the baseline and restoration scenarios:  
- days with stream-floodplain connection in wet (2017, 2019) vs dry years
- the increase in days with connection from baseline to restoration and increase in depth (on average, percentage?)
- increase in total recharge on average
- increase in depth for number of days in wet year

In [ ]:
def fld_chk(lak_out0):
    lak_out0['wy'] = lak_out0.index.year
    lak_out0.loc[lak_out0.index.month>=10,'wy']+=1
    fld_chk = lak_out0.copy()[['GW-Outflw', 'SW-Inflw','depth','wy']]
    # fld_chk['fld'] = fld_chk.depth> 0 # only consider depths greater than 1 ft, Salmon passage?
    fld_chk['fld'] = fld_chk.depth> 1*0.3048 # only consider depths greater than 1 ft, Salmon passage?
    fld_chk['sw_in'] = fld_chk['SW-Inflw']>0 # any inflow
    fld_chk[fld_chk==0] = np.nan
    fld_chk_out =fld_chk.groupby('wy').sum()[['fld','sw_in']]
    mean_chk = fld_chk.groupby('wy').mean()[['GW-Outflw', 'SW-Inflw','depth']]
    return pd.concat((fld_chk_out, mean_chk),axis=1)

In [ ]:
# lak_out0

In [ ]:
# comparison of the number of days
# fld_chk(lak_out0), fld_chk(lak_out)
fld_chk(lak_out0).loc[[2015,2016,2018,2020]].mean()[['fld','sw_in']], fld_chk(lak_out0).loc[[2017, 2019]].mean()[['fld','sw_in']]

In [ ]:
fld_chk(lak_out).loc[[2015,2016,2018,2020]].mean(), fld_chk(lak_out).loc[[2017, 2019]].mean()

In [ ]:

gw_rch_inc = (fld_chk(lak_out0).mean()['GW-Outflw']- fld_chk(lak_out).mean()['GW-Outflw'])/fld_chk(lak_out).mean()['GW-Outflw']
print('Average increase in gw recharge is %.1f %%' %(100*gw_rch_inc))

# impact on depth and recharge
fld_chk(lak_out0).mean()['depth'], fld_chk(lak_out).mean()['depth']

# Identify location and timing of active ET 
Because it isn't easy to map ET, we can do an alternative mapping with head to show it is above the rooting depth.

In [ ]:
# load cbb file from any model scneario to get listing of kstpkper
cbc = join(m.model_ws, 'MF.cbc')

# zon_mod = np.ones((nrow,ncol),dtype=int)
# zb = flopy.utils.ZoneBudget(cbc, zon_mod)
# kstpkper = zb.kstpkper


In [ ]:
et_surf = m.evt.surf.array[0,0]
ext_dp = m.evt.exdp.array[0,0]
# bottom elevation of roots
et_botm =et_surf - ext_dp

et_row, et_col = np.where(ext_dp>2)

In [ ]:
et_lay = get_layer_from_elev(et_botm[et_row, et_col], m.dis.botm[:, et_row, et_col], m.dis.nlay)
# et_lay

In [ ]:
head = hdobj.get_data(dt_ref.kstpkper.iloc[0])
head = np.ma.masked_where(head==-999.99, head)

In [ ]:
def find_active_ET(model_ws):
    hdobj = flopy.utils.HeadFile(join(model_ws, 'MF.hds'))
    et_act = np.zeros((m.dis.nper, nrow,ncol))
    
    for t in np.arange(0,m.dis.nper):
        head = hdobj.get_data(dt_ref.kstpkper.iloc[t])
        head = np.ma.masked_where(head==-999.99, head)
        # identify which GDE ET cells would active based on head
        b = head[et_lay, et_row, et_col] > et_botm[et_row, et_col]
        et_act[t, et_row[b], et_col[b]] = 1
    return et_act


In [ ]:
et_act = find_active_ET(model_ws)

In [ ]:
et_act0 = find_active_ET(model_ws0)

There is very little difference in the spatial location of ET between the scenarios, there are a few scattered dots around the edges of the GDE ET sites that would occur under the baseline scenario but not the no reconnection.  

What is more interesting is the number of days that are active is different. In general there tends to be 20% of the number of days more active ET in the baseline on the western side of oneto denier. The change is larger on the western side because the eastern side of the reconnected floodplain has the recharge due to the river so it is the western side that is dependent on floodplain inundation.  

The spots with greatest ET rates are the drainage areas (not because there is more simulated water there which in reality there is) but because the rooting depth of the GW ET is deeper than elsewhere.

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(9,3), sharey=True, layout='constrained')
im = ax[0].imshow(et_act.mean(axis=0))
plt.colorbar(im, shrink=0.5)# plt.show()

loc_diff = (et_act0.mean(axis=0)>0).astype(int)-(et_act.mean(axis=0)>0).astype(int)
im = ax[1].imshow(loc_diff)
plt.colorbar(im, shrink=0.5)
# plt.colorbar()
# plt.show()

im = ax[2].imshow((et_act0- et_act ).mean(axis=0))
plt.colorbar(im, shrink=0.5)